In [1]:
# imports
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal
import pandas as pd
from glob import glob as glob
import os
from neo import io
import seaborn as sb
from sklearn import preprocessing
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams["font.size"]= 12
plt.rcParams["font.family"] = "Arial"


 

Load preprocessed data and extract sequence variables

In [2]:
# parameters
animal_id = 'A11' # A11, H07
exp = 'classical' # 'wang or classical
out_path = '/Volumes/Bellet/PhD/Local_Global/processedData/%s/'%animal_id # path to the data

# load preprocessed data
df = pd.read_pickle(os.path.join(out_path,'%s_stims_spikes_dataframe.pkl'%(animal_id)))

# extract basic variables
dates = np.unique(df.date)
ndates = len(dates)
nitems = np.max(df.ItemID)
stimID = np.unique(df.StimID)
nstim = len(stimID)
print(list(df.keys()))
print('Recording dates:',dates)

soa = 600
Local = []
Global = []
BlockID = []
Blocktype = []
Block = [] # which block
Include = []
Stimon = []
SessionDate = []
StimID = []
sess = 0
blocks = ['aa','aB','bb','bA'] # coded like this in the experiment
for d,date in enumerate(dates):


    
    # get conditions
    last_item = np.where((df.ItemID==3) & (df.date==date) )[0]#& ((df.StimDur+df.ISIDur)==soa))[0]
    Ntrials = len(last_item)
    if Ntrials>0:
        
        print('Day:',date)
        SessionDate.append(date)
        Local.append(np.zeros(Ntrials).astype(int))
        for i,ind in enumerate(last_item):
            if df.StimID.iloc[ind]!=df.StimID.iloc[ind-1]: # local deviant = transition of two stim
                Local[sess][i] = 1
        
        BlockID.append(np.array(df.blockID[last_item])) # blockID
        Block.append(np.array(df.blockType[last_item])) # which block (classical experiment: 0,1,2 or 3)
        Blocktype.append(np.zeros(Ntrials).astype(int)) # xx=0 xY=1
        for bl in np.unique(BlockID[sess]):
            ind = np.where(BlockID[sess]==bl)[0]
            Blocktype[sess][ind] = Local[sess][ind[0]] # blocktype defined by the first habituation trial in block
        
        Global.append((Local[sess]!=Blocktype[sess]).astype(int))
        
        # filter habituation trials
        nhab = 50
        Include.append(np.ones(len(Global[sess])).astype(int))
        for bl in np.unique(BlockID[sess]):
            ind = np.where(BlockID[sess]==bl)[0]
            Include[sess][ind[:nhab]] = 0
            
        Stimon.append(np.array(df.StimOn.iloc[last_item])) # last stim onset times
        StimID.append(np.array(df.StimID.iloc[last_item]))

        
        sess += 1
    
Session = np.concatenate([np.ones(len(Local[i]))*i for i in range(len(Local))])
Local = np.concatenate(Local)
Global = np.concatenate(Global)
Blocktype = np.concatenate(Blocktype)
Block = np.concatenate(Block)
Include = np.concatenate(Include)
StimID = np.concatenate(StimID)

['PFC_MU', 'PPC_MU', 'PFC_SU', 'PPC_SU', 'TrialID', 'ItemID', 'StimID', 'StimName', 'StimOn', 'blockID', 'blockType', 'date', 'StimDur', 'ISIDur', 'RewardOn']
Recording dates: ['20200226' '20200228' '20200305' '20200306' '20200311' '20200312']
Day: 20200226
Day: 20200228
Day: 20200305
Day: 20200306
Day: 20200311
Day: 20200312


load PSTH

In [3]:

# load PSTH
MUA = []
sess = 0
for d,date in enumerate(dates):
    # get conditions
    last_item = np.where((df.ItemID==3) & (df.date==date) )[0]#& ((df.StimDur+df.ISIDur)==soa))[0]
    Ntrials = len(last_item)
    if Ntrials>0:
        
        print('Day:',date)
        
        # whole sequence
        Rb = np.load(os.path.join(out_path,'Rb_seq_%s.npy'%date))
        
        ntr,nch,nbins = Rb.shape
        
        MUA.append(Rb)
        
        sess += 1
        
del Rb

MUA = np.concatenate(MUA)
time_bins = np.load(os.path.join(out_path,'time_bins_seq_%s.npy'%dates[0])) # time vector
binsize = np.round(np.mean(np.diff(time_bins)),3)
time_bins = time_bins + binsize

Day: 20200226
Day: 20200228
Day: 20200305
Day: 20200306
Day: 20200311
Day: 20200312


## Responsiveness

In [4]:
# use t-test to get p-value
from scipy.stats import ttest_ind, ttest_rel, ranksums
from statsmodels.stats.multitest import multipletests

alpha = 0.01 # significance thrreshold

soa = 0.6 # soa of data
#### TIME WINDOWS ####
window_fix = np.arange(0, np.argmin(abs(time_bins)))
window_seq = np.arange( int((abs(time_bins[0])+3*soa)/binsize), int((abs(time_bins[0])*2 + 3*soa)/binsize)) # define averaging window to average firing rate

nsess = len(dates); nch = MUA.shape[1]
T_seq = np.zeros((nsess,nch))
P_seq = np.zeros((nsess,nch))
for sess in range(nsess):
    ind = Session==sess
    for ch in range(nch):
        fix_resp =  np.mean(MUA[ind,ch,:][:,window_fix],-1) # average response to stimulus A
        seq_resp =  np.mean(MUA[ind,ch,:][:,window_seq],-1) # average response to stimulus A
        T_seq[sess,ch], P_seq[sess,ch] = ttest_rel(fix_resp,seq_resp)

# run false discovery rate procedure on p values from all sessions and channels together
p_flat = P_seq.flatten()
fdr = multipletests(p_flat, alpha=alpha)
p_corr = fdr[1]
P_seq = np.reshape(p_corr,(nsess,nch)) # put back into original matrix


## AUC
Determine bin with maximum performance

In [5]:
AUC_stim = np.load(os.path.join(out_path,'AUC_Stim_first3_classical_600.npy'))
AUC_ctx = np.load(os.path.join(out_path,'AUC_Block_first3_classical_600.npy'))
AUC_loc = np.load(os.path.join(out_path,'AUC_Local_last_True_classical_600.npy'))
AUC_glob = np.load(os.path.join(out_path,'AUC_Global_last_True_classical_600.npy'))
maxbin_stim = np.argmax(np.mean(np.diagonal(AUC_stim[:,:,0,:]),1))
maxbin_ctx = np.argmax(np.mean(np.diagonal(AUC_ctx),1))

In [9]:
# load coefficients
Coef = np.load(os.path.join(out_path,'Coef_last_classical_600.npy'))
RandCoef = np.load(os.path.join(out_path,'CoefRand_last_classical_600.npy'))
Coef_seq = np.load(os.path.join(out_path,'Coef_first3_classical_600.npy'))
RandCoef_seq = np.load(os.path.join(out_path,'CoefRand_first3_classical_600.npy'))
ItemCoef = np.load(os.path.join(out_path,'ItemNumber_Coef.npy'))
RandItemCoef = np.load(os.path.join(out_path,'ItemNumber_CoefRand.npy'))

## Recanatesi et al. Participation Ratio

In [10]:
nperm = RandCoef.shape[0]

In [11]:
loc_ax = 0
glob_ax = 1
stim_ax = 0
ctx_ax = 1

PRloc = np.zeros(nsess)
PRglob = np.zeros(nsess)
PRstim = np.zeros(nsess)
PRctx = np.zeros(nsess)
PRitem = np.zeros((4,nsess))

PRperm = np.zeros((nperm,nsess))

PRsparse = np.zeros(nsess)
PRdist = np.zeros(nsess)

T = Coef.shape[-1] # Number of repeats

for sess in range(nsess):
    # get bin of maximum decoding performance
    max_loc = np.argmax(np.diagonal(AUC_loc[sess,sess,0,]))
    max_glob = np.argmax(np.diagonal(AUC_loc[sess,sess,0,]))
    
    includeChannels = P_seq[sess,:]<alpha
    
    # PR for local axis
    M = Coef[sess,loc_ax,includeChannels,max_loc,:].T
    E = np.cov(M.T)
    PRloc[sess] = np.trace(E)**2 / np.trace(E**2)
    
    # PR for global axis
    M = Coef[sess,glob_ax,includeChannels,max_glob,:].T
    E = np.cov(M.T)
    PRglob[sess] = np.trace(E)**2 / np.trace(E**2)
    
    # PR for stimulus axis
    M = Coef_seq[sess,stim_ax,includeChannels,:].T
    E = np.cov(M.T)
    PRstim[sess] = np.trace(E)**2 / np.trace(E**2)
    
    # PR for context axis
    M = Coef_seq[sess,ctx_ax,includeChannels,:].T
    E = np.cov(M.T)
    PRctx[sess] = np.trace(E)**2 / np.trace(E**2)
    
    # PR for item number axis
    for k in range(4):
        M = ItemCoef[sess,k,includeChannels,:].T
        E = np.cov(M.T)
        PRitem[k,sess] = np.trace(E)**2 / np.trace(E**2)
    
    
    # simulation:
    
    N = sum(includeChannels==True)
    units = 1
    idx = np.random.randint(0,N,units)
    M = np.zeros((T,N)) #M = np.zeros((T,N))
    M[:,idx] = np.repeat(np.random.rand(T,1),units,1) #M [:,idx] = np.random.rand(T,units)
    #M = np.repeat(M,T,0)
    
    E = np.cov(M.T)
    
    PRsparse[sess] = np.trace(E)**2 / np.trace(E**2)
    
    units = N
    M = np.zeros((T,N)) #M = np.zeros((T,N))
    M = np.repeat(np.random.rand(T,1),units,1) #M [:,idx] = np.random.rand(T,units)
    #M = np.repeat(M,T,0)
    
    E = np.cov(M.T)
    
    PRdist[sess] = np.trace(E)**2 / np.trace(E**2)
    
    # randperm
    for k in range(nperm):   
        # local
        M = RandCoef[k,sess,glob_ax,includeChannels,max_glob,:].T
        E = np.cov(M.T)
        PRperm[k,sess] = np.trace(E)**2 / np.trace(E**2)


In [13]:
ratio = PRglob/np.mean(PRperm,0)

In [14]:

ratio = PRloc/PRdist
print('PR for local axis:',np.mean(ratio))
ratio = PRglob/PRdist
print('PR for global axis:',np.mean(ratio))
ratio = PRstim/PRdist
print('PR for stimulus axis:',np.mean(ratio))
ratio = PRctx/PRdist
print('PR for context axis:',np.mean(ratio))
for k in range(4):
    ratio = PRitem[k,:]/PRdist
    print('PR for item %s:'%(k+1),np.mean(ratio))
plt.show()

PR for local axis: 0.7174884234700447
PR for global axis: 0.7350869743165475
PR for stimulus axis: 0.7295121181248808
PR for context axis: 0.7204500657286976
PR for item 1: 0.7435010134427434
PR for item 2: 0.7240906616304336
PR for item 3: 0.7421479372756061
PR for item 4: 0.7156472452726833
